In [1]:
pip install cvxpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Install the needed libraries
import cvxpy as cp
import pandas as pd
import numpy as np
from pulp import *
import time
import warnings
from pandas.core.common import SettingWithCopyWarning

In [3]:
# Ignore SettingWithCopyWarning 
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [4]:
# Import the data
data = pd.read_excel('data_python_modified.xlsx', sheet_name='data', index_col=0)
real = pd.read_excel('data_python_modified.xlsx', sheet_name='реал', index_col=0)
bandwidth = pd.read_excel('data_python_modified.xlsx', sheet_name='ПС', index_col=0)
match = pd.read_excel('data_python_modified.xlsx', sheet_name='match', index_col=0)

In [5]:
# Create a dict of names on a spreadsheets that we are going to use
names = {"Тариф ж/д" : "t1",
        "Тариф хранение" : "t2",
        "Тариф бренд" : "t3",
        "Тариф ВЛ" : "t4",
        "Плечо, км" : "distance",
        "Дата" : "date",
        "НБ" : "origin",
        "ОУ" : "point",
        "НГ" : "brand",
        "НГ_ПС" : "product",
        "Объем" : "volume"} 

In [6]:
# Rename the names in dataframes
data.rename(columns=names, inplace=True)
real.rename(columns=names, inplace=True)
bandwidth.rename(columns=names, inplace=True)
match.rename(columns=names, inplace=True)

In [7]:
# Function for calculating the tarif
def calculate_tarif(row):
    if row["distance"] <= 50:
        return row["t1"] + row["t2"] + row["t3"] + row["t4"]
    else:
        return row["t1"] + row["t2"] + row["t3"] +  row["t4"] * row["distance"]

In [8]:
# Calculate cost
data['cost'] = data.apply(lambda row: calculate_tarif(row), axis=1)

In [9]:
# Checking dates
dates = pd.DataFrame({"date" : data['date'].unique()})
dates

,date
0,2023-04-01
1,2023-03-01


In [10]:
# Save the first date to the t 
t = dates.iloc[0, 0]

In [11]:
# Subset for the time period
data = data[data['date'] == t]
real = real[real['date'] == t]
bandwidth = bandwidth[bandwidth['date'] == t]
    
# Adjust datatypes
data['origin'] = data['origin'].map(lambda x: int(x.strip('Нефтебаза ')))
data['point'] = data['point'].map(lambda x: int(x.strip('АЗС ')))
real['point'] = real['point'].map(lambda x: int(x.strip('АЗС ')))
bandwidth['origin'] = bandwidth['origin'].map(lambda x: int(x.strip('Нефтебаза ')))

In [12]:
# Create some dictionaries for later usage
match_brand_product = dict(zip(match['brand'], match['product']))
match_brand_number = dict(enumerate(match['brand']))
match_product_number = dict(enumerate(match['product']))

match_product_brand = dict()
for key, value in match_brand_product.items():
    match_product_brand.setdefault(value, list()).append(key)
    
match_number_brand = dict()
for key, value in match_brand_number.items():
    match_number_brand[value] = key
    
match_number_product = dict()
for key, value in match_product_number.items():
    match_number_product[value] = key

In [13]:
# Store origin with infinite supply in a separate list
origins_with_inf_supply = list(bandwidth[bandwidth['volume'] == np.inf]['origin'].unique())

In [14]:
# Function for calculating the tarif
def create_var_Name(row):
    return 'P' + str(int(row["point"])) + 'O' + str(int(row["origin"])) +  '_brand_' + str(int(row["brand_number"]))

In [15]:
# Reorganize the data
columns_to_keep = ['origin', 'point', 'brand', 'product', 'cost']

df = data[columns_to_keep].reset_index().drop('index', axis=1).reset_index()

# Code brand, product
df['brand_number'] = df['brand'].map(lambda x: match_number_brand.get(x))
df['product_number'] = df['product'].map(lambda x: match_number_product.get(x))

# Create a name variable
df['var_Name'] = df.apply(lambda row: create_var_Name(row), axis=1) 

In [16]:
# Create demand df
demand = real[['point', 'brand', 'product', 'volume']]
demand.rename(columns={'volume':'demand'}, inplace=True)
demand['brand_number'] = demand['brand'].map(lambda x: match_number_brand.get(x))
demand['product_number'] = demand['product'].map(lambda x: match_number_product.get(x))

In [17]:
# Create supply df
supply = bandwidth[['origin', 'product', 'volume']]
supply.rename(columns={'volume':'supply'}, inplace=True)
supply['product_number'] = supply['product'].map(lambda x: match_number_product.get(x))

In [18]:
df

,index,origin,point,brand,product,cost,brand_number,product_number,var_Name
0,0,0,0,Бензин 100 бренд,АИ 100,3268.488947,0,0,P0O0_brand_0
1,1,0,0,Бензин 92,Аи 92,3268.488947,1,1,P0O0_brand_1
2,2,0,0,Бензин 95,Аи 95,3268.488947,2,3,P0O0_brand_2
3,3,0,0,Бензин 95 бренд,Аи 95,3338.651597,3,3,P0O0_brand_3
4,4,0,0,Топливо дизельное с присадками летнее,ДТ,3133.526439,4,4,P0O0_brand_4
...,...,...,...,...,...,...,...,...,...
355,355,1,47,Топливо дизельное с присадками летнее,ДТ,4734.995826,4,4,P47O1_brand_4
356,356,1,48,Бензин 92,Аи 92,4808.465100,1,1,P48O1_brand_1
357,357,1,48,Бензин 95,Аи 95,4808.465100,2,3,P48O1_brand_2
358,358,1,48,Бензин 95 бренд,Аи 95,4854.005100,3,3,P48O1_brand_3


### experiments

y1^2 + y1^2 == 1

In [135]:
x = cp.Variable(len(df), nonneg = True)

In [136]:
x

Variable((360,), nonneg=True)

In [137]:
c = np.array(df['cost'])

In [138]:
objective = cp.Minimize(c.T @ x)

In [139]:
objective

Minimize(Expression(AFFINE, NONNEGATIVE, ()))

In [140]:
n_constraint = 1
brands = df['brand_number'].unique()

In [141]:
constraints = []

In [142]:
for b in brands:
    df1 = df[df['brand_number'] == b]
    points = df1['point'].unique()
    
    for p in points:
        df2 = df1[df1['point'] == p]
 
        const = float(demand[(demand['point'] == p) & (demand['brand_number'] == b)]['demand'])

        # Add a restriction on demand == const
        constraints.append(cp.sum(x[df2.index]) == const)
        n_constraint +=1

In [143]:
constraints

[Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT,

In [144]:
# Define percents of loading demanded by management
percents_loading = np.arange(0.5, 1, 0.1)
percent_loading = percents_loading[0]

In [145]:
# Function for dividing supply between different brands
def merge_supply(brand_number, match_brand_product, match_brand_number, match_product_brand, match_number_brand):
    """This function returns a set of brands' numbers that are produced from the same product"""
    brand_name = match_brand_number[brand_number]
    product_name = match_brand_product[brand_name]
    brands_overlap_names = match_product_brand[product_name]
    brands_overlap_numbers = set(match_number_brand.get(item) for item in brands_overlap_names)

    return brands_overlap_numbers

In [146]:
# Set for storing already processed brands 
brands_accounted_for = set()

In [147]:
for b in brands:
    
    if b in brands_accounted_for:
        continue
    else:
        brands_overlap_numbers = merge_supply(b, match_brand_product, match_brand_number, match_product_brand, match_number_brand)
        brands_accounted_for.update(brands_overlap_numbers)
        
        df1 = df[df['brand_number'].isin(brands_overlap_numbers)]
        origins = df1['origin'].unique()
        
        for o in origins:
            
            if o in origins_with_inf_supply:
                continue
            else:
                
                df2 = df1[df1['origin'] == o]
                product_name = match_brand_product[match_brand_number[b]]
                const = float(supply[(supply['origin'] == o) & (supply['product'] == product_name)]['supply'])
                
                # Add a restriction on supply <= upper_const
                constraints.append(cp.sum(x[df2.index]) <= const)
                n_constraint +=1

                # Add a restriction on supply >= lower_const
                const *= percent_loading
                constraints.append(cp.sum(x[df2.index]) >= const)
                n_constraint +=1        

In [148]:
constraints

[Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT,

In [149]:
problem = cp.Problem(objective, constraints)

In [150]:
problem.solve(verbose=True)

                                     CVXPY                                     
                                     v1.3.0                                    
(CVXPY) Mar 18 02:52:21 PM: Your problem has 360 variables, 188 constraints, and 0 parameters.
(CVXPY) Mar 18 02:52:21 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 18 02:52:21 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 18 02:52:21 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 18 02:52:21 PM: Compiling problem (target solver=ECOS).
(CVXPY) Mar 18 02:52:21 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing

84060537.5011059

In [151]:
M = 10**9

In [152]:
# Function for creating dummies names
def create_dummy_Name(row):
    return 'y_P' + str(int(row["point"])) + 'O' + str(int(row["origin"])), int(row["point"]), int(row["origin"])

In [153]:
# Create a name variable
[*o] = df.apply(lambda row: create_dummy_Name(row), axis=1)

In [154]:
df_dummies = pd.DataFrame(o, columns=['dummy_Name', 'point', 'origin'])

In [155]:
df_dummies = df_dummies.reset_index()

In [156]:
df_dummies

,index,dummy_Name,point,origin
0,0,y_P0O0,0,0
1,1,y_P0O0,0,0
2,2,y_P0O0,0,0
3,3,y_P0O0,0,0
4,4,y_P0O0,0,0
...,...,...,...,...
355,355,y_P47O1,47,1
356,356,y_P48O1,48,1
357,357,y_P48O1,48,1
358,358,y_P48O1,48,1


In [157]:
df_dummies = df_dummies.drop_duplicates(subset=['dummy_Name', 'point', 'origin'])

In [158]:
df_dummies

,index,dummy_Name,point,origin
0,0,y_P0O0,0,0
5,5,y_P1O0,1,0
10,10,y_P2O0,2,0
14,14,y_P3O0,3,0
18,18,y_P4O0,4,0
...,...,...,...,...
341,341,y_P44O1,44,1
345,345,y_P45O1,45,1
349,349,y_P46O1,46,1
352,352,y_P47O1,47,1


In [159]:
#### Creating our cvxpy variable of assignments
dummies = cp.Variable(len(df_dummies), boolean=True)

In [160]:
dummies

Variable((98,), boolean=True)

In [161]:
#### Creating our cvxpy variable of assignments
# dummies = cp.Variable(len(df_dummies))

In [162]:
df_dummies.reset_index(inplace=True)

In [163]:
df_dummies.drop(columns=['index', 'level_0'], inplace=True)

In [164]:
df_dummies

,dummy_Name,point,origin
0,y_P0O0,0,0
1,y_P1O0,1,0
2,y_P2O0,2,0
3,y_P3O0,3,0
4,y_P4O0,4,0
...,...,...,...
93,y_P44O1,44,1
94,y_P45O1,45,1
95,y_P46O1,46,1
96,y_P47O1,47,1


In [165]:
points = df['point'].unique()
origins = df['origin'].unique()

for p in points:
    for o in origins:
        df1 = df[(df['point'] == p) & (df['origin'] == o)]
        
        
        df_d = df_dummies[(df_dummies['point'] == p) & (df_dummies['origin'] == o)]
        
        constraints.append(cp.sum(x[df1.index]) <= M * dummies[df_d.index])
       
        n_constraint += 1

In [166]:
len(constraints)

286

In [167]:
const = 1

In [168]:
for p in points:
    
    df_d = df_dummies[df_dummies['point'] == p]

    constraints.append(cp.sum(dummies[df_d.index]) == const)
    
    n_constraint += 1

In [169]:
len(constraints)

335

In [170]:
problem = cp.Problem(objective, constraints)

In [171]:
problem.solve(verbose=True)

                                     CVXPY                                     
                                     v1.3.0                                    
(CVXPY) Mar 18 02:55:28 PM: Your problem has 458 variables, 335 constraints, and 0 parameters.
(CVXPY) Mar 18 02:55:28 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 18 02:55:28 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 18 02:55:28 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 18 02:55:28 PM: Compiling problem (target solver=COPT).
(CVXPY) Mar 18 02:55:28 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing

84451667.32999241

In [84]:
# import coptpy

In [86]:
# import cylp

In [50]:
# problem = cp.Problem(objective, constraints)

In [175]:
# problem.solve(solver=cp.SCIP, verbose=True)

In [174]:
# pip install pyscipopt 

In [173]:
# pip install coptpy

In [172]:
# pip install cylp